In [18]:
import numpy as np
from tqdm import tqdm
from hw2_code.kmeans import KMeans

# data = np.random.randn(5, 4)
# data[:, 1] = np.array((1,0.5,1,-1,0.641))
data = np.array([[0.10782656, 0.04982049, 0.78844897, 0.05390398],
                [0.16080479, 0.70138883, 0.05805256, 0.07975382],
                [0.39637071, 0.23624673, 0.0996817 , 0.26770086],
                [0.21741805, 0.56913777, 0.05890126, 0.15454293],
                [0.27040961, 0.54778227, 0.01886611, 0.16294201]])

# data[4] = np.zeros((1,4))

# data = np.resize(np.arange(20), (5,4))

print(data)

[[0.10782656 0.04982049 0.78844897 0.05390398]
 [0.16080479 0.70138883 0.05805256 0.07975382]
 [0.39637071 0.23624673 0.0996817  0.26770086]
 [0.21741805 0.56913777 0.05890126 0.15454293]
 [0.27040961 0.54778227 0.01886611 0.16294201]]


In [92]:
### Helper functions
"""
logit is N x D
"""
logit = data

## softmax
def softmax(logit):
    prob = np.exp(logit - np.max(logit, axis=-1, keepdims=True)) / np.sum(np.exp(logit - np.max(logit, axis=-1, keepdims=True)), axis=-1, keepdims=True)
    return prob

prob = softmax(logit)

## logsumexp
def logsumexp(logit):
    s = np.log(np.sum(np.exp(logit - np.max(logit, axis=-1, keepdims=True)), axis=-1, keepdims=True)) + np.max(logit, axis=-1, keepdims=True)
    return s

s = logsumexp(logit)

## multinormalPDF

SIGMA_CONST = 1e-6
mu_i = data.mean(axis=0)
sigma_i = np.cov(data, rowvar=False)
points = data

def multinormalPDF(points, mu_i, sigma_i):
    D = mu_i.shape[0]
    try:
        inv = np.linalg.inv(sigma_i)
    except np.linalg.LinAlgError:
        inv = np.linalg.inv(sigma_i + SIGMA_CONST)

    semi = (points - mu_i) @ inv
    NN =  np.sum((-0.5 * (semi.T * (points - mu_i).T)), axis=0)
    normal_pdf = (1/((2*np.pi) ** (D/2))) * (np.linalg.det(sigma_i) ** (-0.5)) * np.exp(NN)
    return normal_pdf

multinormalPDF(points, mu_i, sigma_i)

array([1.16114174e+14, 5.97440918e+10, 5.83938679e+10, 1.08309693e+10,
       4.43424352e+10])

In [128]:
### Initialization and log-likelihood

N = data.shape[0]
D = data.shape[1]
K = 3

## __init_components

def _init_components(K, D, N, data):

    np.random.seed(5) #Do Not Remove Seed

    pi = np.ones((K,)) / K
    mu = np.ones((K, D))
    for i in range(K):
        mu[i] = data[int(np.random.uniform(0, N-1))] # sei lá pq N-1, mas eu tentei do nadão e funcionou
    sigma = np.ones((K, D, D)) * np.eye(D)

    return pi, mu, sigma

pi, mu, sigma = _init_components(K, D, N, data)


## _ll_joint
FULL_MATRIX = True
LOG_CONST = 1e-32

def _ll_joint(pi, mu, sigma, full_matrix=FULL_MATRIX, points=points, **kwargs):
# === graduate implementation
    if full_matrix is True:
        ll = np.ones((N, K))
        for i in range(N): #### TEM COMO EVITAR ESSE LOOP?
            for k in range(K):
                pdf = multinormalPDF(points[i], mu[k], sigma[k])
                ll[i, k] = np.log( pi[k] + LOG_CONST ) + np.log( pdf + LOG_CONST )

    return ll


In [296]:
### Expectation Maximization

## E-step

def _E_step(pi, mu, sigma, full_matrix = FULL_MATRIX , **kwargs):
# === graduate implementation
    if full_matrix is True:
        gamma = softmax(_ll_joint(pi, mu, sigma, full_matrix=FULL_MATRIX))

    return gamma

gamma = _E_step(pi, mu, sigma)

## M-step

def _M_step(gamma, full_matrix=FULL_MATRIX, **kwargs):
# === graduate implementation
    if full_matrix is True:
        N_k = gamma.sum(axis=0)

        pi_new = N_k / points.shape[0]

        mu_new = ( (gamma.T @ points).T / N_k ).T  # ATÉ AQUI TÁ CERTO!

        diff = np.ones((K, N, D))
        sigma_new = np.ones((K, D, D))
        for i in range(K):
            diff = points - mu_new[i]
            sigma_new[i] = ( np.dot(gamma[:,i].T * diff.T, diff) ) / N_k[i]

    return pi_new, mu_new, sigma_new

_M_step(gamma)


(array([0.30522247, 0.38955507, 0.30522247]),
 array([[0.22575676, 0.39480348, 0.23907978, 0.14035998],
        [0.23810208, 0.46173044, 0.15105716, 0.14911032],
        [0.22575676, 0.39480348, 0.23907978, 0.14035998]]),
 array([[[ 0.01040283,  0.00195728, -0.02010223,  0.00774211],
         [ 0.00195728,  0.06145035, -0.06441879,  0.00101115],
         [-0.02010223, -0.06441879,  0.09916263, -0.01464162],
         [ 0.00774211,  0.00101115, -0.01464162,  0.00588835]],
 
        [[ 0.00885635, -0.00435757, -0.01112552,  0.00662674],
         [-0.00435757,  0.04911154, -0.041034  , -0.00371997],
         [-0.01112552, -0.041034  ,  0.06017613, -0.00801662],
         [ 0.00662674, -0.00371997, -0.00801662,  0.00510985]],
 
        [[ 0.01040283,  0.00195728, -0.02010223,  0.00774211],
         [ 0.00195728,  0.06145035, -0.06441879,  0.00101115],
         [-0.02010223, -0.06441879,  0.09916263, -0.01464162],
         [ 0.00774211,  0.00101115, -0.01464162,  0.00588835]]]))